In [40]:
import pandas as pd
import numpy as np
import openfoodfacts
import geopandas
import geopy

The user scans the barcode and the system gets the number. Here done via manual input

In [41]:
barcode = input('get the barcode: ')

get the barcode: 3760094310634


Search for the product

In [42]:
product = openfoodfacts.products.get_product(barcode)

In [43]:
df = pd.DataFrame(zip(product['product'].keys(), product['product'].values()), columns=['tags','value'])

In [44]:
df

,tags,value
0,rev,31
1,lang_imported,fr
2,ingredients_analysis_tags,"[en:palm-oil-free, en:maybe-vegan, en:vegetarian]"
3,unknown_nutrients_tags,[]
4,traces_lc,fr
...,...,...
246,unknown_ingredients_n,0
247,generic_name_fr,céréales de petit déjeuner aux flocons d'avoin...
248,origins_tags,[]
249,brands_imported,Aline & Olivier


We get the only relevant data

In [45]:
df = df.T
df.columns = df.iloc[0]
df = df[1:]
campos = df[['cities_tags', 'emb_codes', 'manufacturing_places']]
campos

tags,cities_tags,emb_codes,manufacturing_places
value,[],DDM,13103 SAINT ETIENNE DU GRES


Search for the place of origin from those fields.  
It is underdstood that the city field is the most appropriate, then the tracability code then the manufacturing places.  
If those information are empty, the place of origin cannot be displayed.

In [46]:
place_of_origin = ''

if 'cities_tags' in campos.columns and len(campos['cities_tags'][0]) != 0:
    place_of_origin = campos['cities_tags'][0]
    
#elif 'emb_codes' in campos.columns and len(campos['emb_codes'][0]) != 0:
#    search_result = openfoodfacts.products.search(campos['emb_codes'][0])
#    if 'manufacturing_places' in search_result['products'] and len(search_result['products'][0]['manufacturing_places']) != 0:
#        place_of_origin = search_result['products'][0]['manufacturing_places'][0]
        
elif 'manufacturing_places' in campos.columns and len(campos['manufacturing_places']) != 0:
    place_of_origin = campos['manufacturing_places'][0]
    
else:
    place_of_origin = 'No known origin'

In [47]:
place_of_origin

'13103 SAINT ETIENNE DU GRES'

In [48]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(place_of_origin)

In [49]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 43.7809809, Longitude = 4.7311849
